<a href="https://colab.research.google.com/github/shubham9875/EDAProjectFinal/blob/main/EDAProjectFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
pf=pd.read_csv("/content/drive/MyDrive/eda proj/train 2.csv")

In [26]:
pf.head(3)

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422


In [27]:
pf.shape

(550068, 12)

In [28]:
pf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 12 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   User_ID                     550068 non-null  int64  
 1   Product_ID                  550068 non-null  object 
 2   Gender                      550068 non-null  object 
 3   Age                         550068 non-null  object 
 4   Occupation                  550068 non-null  int64  
 5   City_Category               550068 non-null  object 
 6   Stay_In_Current_City_Years  550068 non-null  object 
 7   Marital_Status              550068 non-null  int64  
 8   Product_Category_1          550068 non-null  int64  
 9   Product_Category_2          376430 non-null  float64
 10  Product_Category_3          166821 non-null  float64
 11  Purchase                    550068 non-null  int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 50.4+ MB


In [29]:
pf.columns

Index(['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category',
       'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1',
       'Product_Category_2', 'Product_Category_3', 'Purchase'],
      dtype='object')

In [30]:
pf.rename(columns={'User_ID':'u_id','Product_ID':'p_id','Gender':'gender', 'Age':'age','Occupation':'occupation',
                    'City_Category':'city_category','Stay_In_Current_City_Years':'years', 'Marital_Status':'marital',
                     'Product_Category_1': 'product1','Product_Category_2':'product2', 'Product_Category_3':'product3',
                     'Purchase':'purchase'},inplace = True)


In [31]:
pf.isnull().sum()

u_id                  0
p_id                  0
gender                0
age                   0
occupation            0
city_category         0
years                 0
marital               0
product1              0
product2         173638
product3         383247
purchase              0
dtype: int64

In [32]:
pf.duplicated().sum()

0

In [33]:
category = pf.loc[:,['product1', 'product2', 'product3']]
category.describe()

,product1,product2,product3
count,550068.000000,376430.000000,166821.000000
mean,5.404270,9.842329,12.668243
std,3.936211,5.086590,4.125338
min,1.000000,2.000000,3.000000
25%,1.000000,5.000000,9.000000
50%,5.000000,9.000000,14.000000
75%,8.000000,15.000000,16.000000
max,20.000000,18.000000,18.000000


In [34]:
# Droping Product id

In [35]:
# Here we are dropping product_id column as there is no need of that columns in data 
pf.drop(['p_id'],inplace = True, axis = 'columns')

In [36]:
pf.head(3)

,u_id,gender,age,occupation,city_category,years,marital,product1,product2,product3,purchase
0,1000001,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,F,0-17,10,A,2,0,12,NaN,NaN,1422


In [37]:
# we dont need + in 4+ years in city column
# so we will remove them 

pf['years'].unique()

array(['2', '4+', '3', '1', '0'], dtype=object)

In [38]:
def cities(value):
    if '+' in value:
        value = value.replace('+','')
        return int(value)
    else:
        return int(value)

pf['years'] = pf['years'].apply(cities)

In [39]:
pf['years'].unique()

array([2, 4, 3, 1, 0])

In [42]:
# User_id
pf['u_id'].unique()

array([1000001, 1000002, 1000003, ..., 1004113, 1005391, 1001529])

In [44]:
pf['u_id'].dtype

dtype('int64')

In [45]:
#genders
pf['gender'].unique()

array(['F', 'M'], dtype=object)

In [46]:
pf['gender'].dtype

dtype('O')

In [47]:
# Age
pf['age'].unique()

array(['0-17', '55+', '26-35', '46-50', '51-55', '36-45', '18-25'],
      dtype=object)

In [50]:
pf.head(3)

,u_id,gender,age,occupation,city_category,years,marital,product1,product2,product3,purchase
0,1000001,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,F,0-17,10,A,2,0,12,NaN,NaN,1422


In [51]:
# As age is present in range so now we convert the range into particular name like

# 0-17 -> child
# 18-25 -> teenage
# 26-35 -> adult
# 36-45 -> adult
# 46-50 -> adult
# 51-55 -> old
# 55+ -> old

# we have replaced numerical data into categorical data


def age(value):
    if '0-17' in value:
        value = value.replace('0-17','child')
        return str(value)
    elif '18-25' in value:
        value = value.replace('18-25','teenage')
        return str(value)
    elif '26-35' in value:
        value = value.replace('26-35','adult')
        return str(value)
    elif '36-45' in value:
        value = value.replace('36-45','adult')
        return str(value)
    elif '46-50' in value:
        value = value.replace('46-50','adult')
        return str(value)
    elif '51-55' in value:
        value = value.replace('51-55','old')
        return str(value)
    elif '55+' in value:
        value = value.replace('55+','old')
        return str(value)
pf['age'] = pf['age'].apply(age)

# changes are also done in inp1 using apply method
        

In [57]:
pf.head(10)

,u_id,gender,age,occupation,city_category,years,marital,product1,product2,product3,purchase
0,1000001,F,child,10,A,2,0,3,NaN,NaN,8370
1,1000001,F,child,10,A,2,0,1,6.0,14.0,15200
2,1000001,F,child,10,A,2,0,12,NaN,NaN,1422
3,1000001,F,child,10,A,2,0,12,14.0,NaN,1057
4,1000002,M,old,16,C,4,0,8,NaN,NaN,7969
5,1000003,M,adult,15,A,3,0,1,2.0,NaN,15227
6,1000004,M,adult,7,B,2,1,1,8.0,17.0,19215
7,1000004,M,adult,7,B,2,1,1,15.0,NaN,15854
8,1000004,M,adult,7,B,2,1,1,16.0,NaN,15686
9,1000005,M,adult,20,A,1,1,8,NaN,NaN,7871


In [58]:
pf['age'].unique()

array(['child', 'old', 'adult', 'teenage'], dtype=object)